<a href="https://colab.research.google.com/github/Linux-Server/DeepLearning.AI/blob/main/Revision_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline

In [3]:
classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
classifier("I love you")

[{'label': 'positive', 'score': 0.8594695329666138}]

In [29]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
raw_inputs = ["I love you", "Look into my eyes, you soul is stained by the blood of innocents"]

outputs = tokenizer(raw_inputs, truncation=True, padding=True, return_tensors="pt")

outputs


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[    0,   100,   657,    47,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1],
        [    0, 15833,    88,   127,  2473,     6,    47,  7047,    16, 31789,
            30,     5,  1925,     9, 40374,  4189,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [30]:
from transformers import AutoModel, AutoModelForSequenceClassification

model = AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
virtual = model(**outputs)
virtual.last_hidden_state.shape


torch.Size([2, 17, 768])

In [31]:
sequence_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
real = sequence_model(**outputs)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
real.logits.shape

torch.Size([2, 3])

In [32]:
model.config.id2label


{0: 'negative', 1: 'neutral', 2: 'positive'}

In [33]:
import torch

predictions = torch.nn.functional.softmax(real.logits, dim=-1)
print(predictions)

tensor([[0.0181, 0.1224, 0.8595],
        [0.7859, 0.1977, 0.0164]], grad_fn=<SoftmaxBackward0>)
